In [ ]:
pip install -r requirements.txt

In [19]:
import os
import json
from coinbase_agentkit import (
    AgentKit,
    AgentKitConfig,
    CdpEvmServerWalletProvider,
    CdpEvmServerWalletProviderConfig,
    allora_action_provider,
    cdp_api_action_provider,
    erc20_action_provider,
    pyth_action_provider,
    wallet_action_provider,
    weth_action_provider,
    Action
)
from dotenv import load_dotenv
import time
import functools
import inspect
from typing import Any, Callable, Dict, List
import sys
from strands.models import BedrockModel
from strands import Agent

In [20]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [21]:
# Set Wallet credetials as Environment Variables
os.environ["CDP_API_KEY_ID"] = "API KEY"
os.environ["CDP_API_KEY_SECRET"] = "API SECRET KEY"
os.environ["CDP_WALLET_SECRET"] = "WALLET SECERT"

# Configure network and file path
network_id = os.getenv("NETWORK_ID", "base-sepolia")
wallet_file = f"wallet_data_{network_id.replace('-', '_')}.txt"

# Load existing wallet data if available
wallet_data = {}
if os.path.exists(wallet_file):
    try:
        with open(wallet_file) as f:
            wallet_data = json.load(f)
            print(f"Loading existing wallet from {wallet_file}")
    except json.JSONDecodeError:
        print(f"Warning: Invalid wallet data for {network_id}")
        wallet_data = {}

# Determine wallet address using priority order
wallet_address = (
    wallet_data.get("address")  # First priority: Saved wallet file
    or os.getenv("ADDRESS")  # Second priority: ADDRESS env var
    or None  # Will trigger idempotency flow if needed
)

wallet_provider = CdpEvmServerWalletProvider(
        CdpEvmServerWalletProviderConfig(
            api_key_id=os.getenv("CDP_API_KEY_ID"),  # CDP API Key ID
            api_key_secret=os.getenv("CDP_API_KEY_SECRET"),  # CDP API Key Secret
            wallet_secret=os.getenv("CDP_WALLET_SECRET"),  # CDP Wallet Secret
            network_id=network_id,  # Network ID - Optional, will default to 'base-sepolia'
            address=wallet_address,  # Wallet Address - Optional, will trigger idempotency flow if not provided
            idempotency_key=(os.getenv("IDEMPOTENCY_KEY") if not wallet_address else None),  # Idempotency Key - Optional, seeds generation of a new wallet
        )
    )


# Create AgentKit instance with wallet and action providers
agentkit = AgentKit(
        AgentKitConfig(
            wallet_provider=wallet_provider,
            action_providers=[
                cdp_api_action_provider(),
                erc20_action_provider(),
                pyth_action_provider(),
                wallet_action_provider(),
                weth_action_provider(),
                allora_action_provider(),
            ],
        )
    )

new_wallet_data = {
        "address": wallet_provider.get_address(),
        "network_id": network_id,
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S")
        if not wallet_data
        else wallet_data.get("created_at"),
}

with open(wallet_file, "w") as f:
    json.dump(new_wallet_data, f, indent=2)
    print(f"Wallet data saved to {wallet_file}")

Loading existing wallet from wallet_data_base_sepolia.txt
Wallet data saved to wallet_data_base_sepolia.txt


In [22]:
"""Strands integration tools for AgentKit."""
def _generate_docstring_from_schema(action: Action) -> str:
    """Generate a docstring from an action's schema."""
    schema = action.args_schema.model_json_schema()

    # Start with the action description
    docstring = f"{action.description}\n\n"

    # Add Args section if there are properties
    if "properties" in schema and schema["properties"]:
        docstring += "Args:\n"
        for prop_name, prop_info in schema["properties"].items():
            desc = prop_info.get("description", "")
            docstring += f"    {prop_name}: {desc}\n"

    return docstring


def _infer_type_hints(action: Action) -> Dict[str, Any]:
    """Infer Python type hints from schema types."""
    schema = action.args_schema.model_json_schema()
    type_hints = {}

    if "properties" in schema:
        for prop_name, prop_info in schema["properties"].items():
            # Simple type mapping - expand as needed
            json_type = prop_info.get("type")
            if json_type == "string":
                type_hints[prop_name] = str
            elif json_type == "integer":
                type_hints[prop_name] = int
            elif json_type == "number":
                type_hints[prop_name] = float
            elif json_type == "boolean":
                type_hints[prop_name] = bool
            elif json_type == "object":
                type_hints[prop_name] = dict
            elif json_type == "array":
                type_hints[prop_name] = list
            else:
                type_hints[prop_name] = Any

            # Handle anyOf/oneOf cases
            if "anyOf" in prop_info or "oneOf" in prop_info:
                type_hints[prop_name] = Any

    return type_hints


def create_strands_tool(action: Action) -> Callable:
    """Create a Strands tool function from an AgentKit action."""

    # Get type hints and parameter info from the schema
    type_hints = _infer_type_hints(action)

    # Define a generic function that will become our template
    def template_function(*args, **kwargs):
        # This function will be replaced with a properly-parameterized version
        pass

    # Create parameters for our new function signature
    parameters = []
    for param_name in type_hints.keys():
        param = inspect.Parameter(
            name=param_name,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            annotation=type_hints.get(param_name, inspect.Parameter.empty)
        )
        parameters.append(param)

    # Create a new signature with these parameters
    new_signature = inspect.Signature(parameters, return_annotation=dict)

    # Create the action handler with proper parameter handling
    def action_handler(*args, **kwargs):
        try:
            # Convert positional args to named args
            param_names = list(type_hints.keys())
            all_kwargs = kwargs.copy()
            for i, arg in enumerate(args):
                if i < len(param_names):
                    all_kwargs[param_names[i]] = arg
    
            # Invoke the action with all arguments
            result = action.invoke(all_kwargs)
            return {
                "status": "success",
                "content": [{"text": result}]
            }
        except Exception as e:            
            return {
                "status": "error",
                 "content": [
                    {"text": f"Error:{e}"}
                ]
            }

    # Clone our template function
    tool_function = functools.update_wrapper(action_handler, template_function)

    # Update the function's signature, name and docstring
    tool_function.__signature__ = new_signature
    tool_function.__name__ = action.name
    tool_function.__doc__ = _generate_docstring_from_schema(action)
    tool_function.__annotations__ = {**type_hints, "return": dict}

    # Apply the @tool decorator
    decorated_func = tool(name=action.name)(tool_function)

    return decorated_func


def get_strands_tools(agent_kit: AgentKit) -> List[Callable]:
    """Get Strands tools from an AgentKit instance.

    Args:
        agent_kit: The AgentKit instance

    Returns:
        A list of Strands tool functions
    """
    actions: List[Action] = agent_kit.get_actions()

    tools = []
    for action in actions:
        strands_tool = create_strands_tool(action)
        tools.append(strands_tool)

    return tools

In [23]:
# Convert AgentKit actions to Strands tools
strands_tools = get_strands_tools(agentkit)

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name='us-east-1'
)

# Create a Strands agent with the converted tools
agent = Agent(
    model=bedrock_model,
    tools=strands_tools,
    system_prompt = """"You are a helpful agent that can interact onchain using the Coinbase Developer Platform AgentKit.
You are empowered to interact onchain using your tools. If you ever need funds, you can request them from the faucet if you are on network ID 'base-sepolia'. 
If not, you can provide your wallet details and request funds from the user. Before executing your first action, get the wallet details to see what network you're on. 
If there is a 5XX (internal) HTTP error code, ask the user to try again later. 
If someone asks you to do something you can't do with your currently available tools, you must say so, and encourage them to implement it themselves using the CDP SDK + Agentkit, recommend they go to docs.cdp.coinbase.com for more information. 
Be concise and helpful with your responses. Refrain from restating your tools' descriptions unless it is explicitly requested.
"""
)

In [ ]:
# Autonomous Mode
async def run_autonomous_mode(agent, interval=10):
    """Run the agent autonomously with specified intervals."""
    print("Starting autonomous mode...")
    while True:
        try:
            thought = "Be creative and do something interesting on the blockchain. Choose an action or set of actions and execute it that highlights your abilities."

            # Run agent in autonomous mode
            print(agent(thought))
            print("-------------------")

            # Wait before the next action
            time.sleep(interval)

        except KeyboardInterrupt:
            print("Goodbye Agent!")
            sys.exit(0)


# Chat Mode
async def run_chat_mode(agent):
    """Run the agent interactively based on user input."""
    print("Starting chat mode... Type 'exit' to end.")
    while True:
        try:
            user_input = input("\nPrompt: ")
            if user_input.lower() == "exit":
                break

            # Run agent with the user's input in chat mode
            print(agent(user_input))
            print("-------------------")

        except KeyboardInterrupt:
            print("Goodbye Agent!")
            sys.exit(0)


# Mode Selection
def choose_mode():
    """Choose whether to run in autonomous or chat mode based on user input."""
    while True:
        print("\nAvailable modes:")
        print("1. chat    - Interactive chat mode")
        print("2. auto    - Autonomous action mode")

        choice = input("\nChoose a mode (enter number or name): ").lower().strip()
        if choice in ["1", "chat"]:
            return "chat"
        elif choice in ["2", "auto"]:
            return "auto"
        print("Invalid choice. Please try again.")


async def main(agent):
    """Start the chatbot agent."""
    # Load environment variables
    load_dotenv()

    # Run the agent in the selected mode
    mode = choose_mode()
    if mode == "chat":
        await run_chat_mode(agent=agent)
    elif mode == "auto":
        await run_autonomous_mode(agent=agent)


if __name__ == "__main__":
    print("Starting Agent...")
    asyncio.run(main(agent))